In [1]:
from ultralytics import YOLO
import ultralytics.data as data
import ultralytics.data.dataset as dataset
import ultralytics.data.build as build

import numpy as np

In [3]:
class YOLOWeightedDataset(data.dataset.YOLODataset):
    def __init__(self, *args, mode="train", **kwargs):
        """
        Initialize the WeightedDataset.

        Args:
            class_weights (list or numpy array): A list or array of weights corresponding to each class.
        """

        super(YOLOWeightedDataset, self).__init__(*args, **kwargs)

        self.train_mode = "train" in self.prefix

        # You can also specify weights manually instead
        self.count_instances()
        class_weights = np.sum(self.counts) / self.counts

        # Aggregation function
        self.agg_func = np.mean

        self.class_weights = np.array(class_weights)
        self.weights = self.calculate_weights()
        self.probabilities = self.calculate_probabilities()

    def count_instances(self):
        """
        Count the number of instances per class

        Returns:
            dict: A dict containing the counts for each class.
        """
        self.counts = [0 for i in range(len(self.data["names"]))]
        for label in self.labels:
            cls = label['cls'].reshape(-1).astype(int)
            for id in cls:
                self.counts[id] += 1

        self.counts = np.array(self.counts)
        self.counts = np.where(self.counts == 0, 1, self.counts)

    def calculate_weights(self):
        """
        Calculate the aggregated weight for each label based on class weights.

        Returns:
            list: A list of aggregated weights corresponding to each label.
        """
        weights = []
        for label in self.labels:
            cls = label['cls'].reshape(-1).astype(int)

            # Give a default weight to background class
            if cls.size == 0:
              weights.append(1)
              continue

            # Take mean of weights
            # You can change this weight aggregation function to aggregate weights differently
            weight = self.agg_func(self.class_weights[cls])
            weights.append(weight)
        return weights

    def calculate_probabilities(self):
        """
        Calculate and store the sampling probabilities based on the weights.

        Returns:
            list: A list of sampling probabilities corresponding to each label.
        """
        total_weight = sum(self.weights)
        probabilities = [w / total_weight for w in self.weights]
        return probabilities

    def __getitem__(self, index):
        """
        Return transformed label information based on the sampled index.
        """
        # Don't use for validation
        if not self.train_mode:
            return self.transforms(self.get_image_and_label(index))
        else:
            index = np.random.choice(len(self.labels), p=self.probabilities)
            return self.transforms(self.get_image_and_label(index))
        
dataset.YOLODataset = YOLOWeightedDataset

In [4]:
import ultralytics.data.build as build

build.YOLODataset = YOLOWeightedDataset

In [ ]:
model = YOLO('yolo11m-cls.pt')  # Load the classification model

# Start training
results = model.train(
    data=r"d:\Kananat\Data\training_dataset_2D\2DLayers\training_dataset_genSclerosis",
    name='genSclerosis_2D_layers',
    epochs=500,
    patience=150,
    imgsz=640,
    dropout=0.1,
    batch=16,
    device='0'  # Use '0' for first GPU, 'cpu' for CPU training
)

New https://pypi.org/project/ultralytics/8.3.203 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.166  Python-3.12.11 torch-2.7.1+cu118 CUDA:0 (Quadro P4000, 8192MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=d:\Kananat\Data\training_dataset_2D\2DLayers\training_dataset_genSclerosis, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11m-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=ge

train: Scanning D:\Kananat\Data\training_dataset_2D\2DLayers\training_dataset_genSclerosis\train... 5100 images, 0 corrupt: 100%|██████████| 5100/5100 [00:00<00:00, 5602.65it/s]


train: New cache created: D:\Kananat\Data\training_dataset_2D\2DLayers\training_dataset_genSclerosis\train.cache
val: Fast image access  (ping: 0.10.0 ms, read: 93.913.4 MB/s, size: 12.8 KB)


val: Scanning D:\Kananat\Data\training_dataset_2D\2DLayers\training_dataset_genSclerosis\val... 1380 images, 0 corrupt: 100%|██████████| 1380/1380 [00:00<00:00, 4076.69it/s]


val: New cache created: D:\Kananat\Data\training_dataset_2D\2DLayers\training_dataset_genSclerosis\val.cache
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 49 weight(decay=0.0), 50 weight(decay=0.0005), 50 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\classify\genSclerosis_2D_layers2
Starting training for 500 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...


      Epoch    GPU_mem       loss  Instances       Size


     65/500      5.86G     0.2612         12        640: 100%|██████████| 319/319 [16:13<00:00,  3.05s/it]
               classes   top1_acc   top5_acc: 100%|██████████| 44/44 [00:59<00:00,  1.34s/it]

                   all      0.831          1



      Epoch    GPU_mem       loss  Instances       Size


     66/500      5.78G     0.2705         16        640:  17%|█▋        | 53/319 [02:41<13:32,  3.06s/it]